# 🏆 Book com Feature Store Consolidada

## Objetivo
Consolidar todas as variáveis disponíveis em um único dataset enriquecido para alimentar modelos de Machine Learning.

## Arquitetura de Dados

```
┌────────────────────────────────────────────────────────────────────────────────────────────────┐
│                          BOOK - FEATURE STORE                                                  │
├────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                                                                                │
│   ┌──────────────────┐                                                                         │
│   │ dados_cadastrais │ ◄── BASE PRINCIPAL (Chave: NUM_CPF + SAFRA)                             │
│   └────────┬─────────┘                                                                         │
│            │                                                                                   │
│   ┌────────┴─────────┐───────────────────┐───────────────────┐─────────────────────┐           │
│   │                  │                   │                   │                     │           │
│   ▼                  ▼                   ▼                   ▼                     ▼           │
│ ┌──────┐    ┌────────────────┐    ┌─────────────┐    ┌──────────────┐    ┌──────────────────┐  │
│ │telco │    │score_bureau_   │    │ass_recarga_ │    │book_recargas_│    │ book_faturamento │  │
│ │      │    │movel (TARGET)  │    │cmv_nova     │    │pagamento     │    │                  │  │
│ └──────┘    └────────────────┘    └─────────────┘    └──────────────┘    └──────────────────┘  │
│                                                                                                │
│                                                                                                │
└────────────────────────────────────────────────────────────────────────────────────────────────┘
```

## Tabelas Fonte

| Tabela | Camada | Tipo | Descrição |
|--------|--------|------|----------|
| dados_cadastrais | Silver | Base | Dados cadastrais e perfil do cliente |
| telco | Silver | Enriquecimento | Variáveis de comportamento telco |
| score_bureau_movel | Silver | **Target** | Scores e variáveis resposta (FPD) |
| ass_recarga_cmv_nova | Book | Features | Book de recargas CMV |
| book_recargas_pagamento | Book | Features | Book de pagamentos |
| book_faturamento | Book | Features | Book de faturamento |

## Destino
- **Lakehouse**: Gold
- **Schema**: feature_store
- **Tabela**: clientes

---
**Versão**: 1.0  
**Granularidade**: NUM_CPF + SAFRA


In [ ]:
# =============================================================================
# CONFIGURAÇÃO DO AMBIENTE (importado do config centralizado)
# =============================================================================
import sys; sys.path.insert(0, "/lakehouse/default/Files/projeto-final")
from config.pipeline_config import (
    SILVER_BASE, GOLD_BASE,
    PATH_DADOS_CADASTRAIS, PATH_TELCO, PATH_SCORE_BUREAU,
    PATH_BOOK_RECARGA_CMV, PATH_BOOK_PAGAMENTO, PATH_BOOK_FATURAMENTO,
    PATH_FEATURE_STORE
)

# Lakehouse paths (para compatibilidade com print abaixo)
LAKEHOUSE_SILVER = SILVER_BASE
LAKEHOUSE_GOLD = GOLD_BASE

# =============================================================================
# PATHS DE SAÍDA (Gold)
# =============================================================================
PATH_OUTPUT = PATH_FEATURE_STORE

print("✅ Configuração carregada com sucesso")
print(f"\n📥 Origem (Silver): {LAKEHOUSE_SILVER.split('/')[-1][:8]}...")
print(f"📤 Destino (Gold): {LAKEHOUSE_GOLD.split('/')[-1][:8]}...")
print(f"\n📁 Output: feature_store.clientes_consolidado")

In [2]:
# =============================================================================
# CARREGAMENTO DAS TABELAS
# =============================================================================

print("📊 Carregando tabelas...\n")

# -----------------------------------------------------------------------------
# TABELA BASE: dados_cadastrais
# -----------------------------------------------------------------------------
df_cadastro = spark.read.format("delta").load(PATH_DADOS_CADASTRAIS)
print(f"✅ dados_cadastrais: {df_cadastro.count():,} registros | {len(df_cadastro.columns)} colunas")

# -----------------------------------------------------------------------------
# TABELAS DE ENRIQUECIMENTO (Raw)
# -----------------------------------------------------------------------------
df_telco = spark.read.format("delta").load(PATH_TELCO)
print(f"✅ telco: {df_telco.count():,} registros | {len(df_telco.columns)} colunas")

df_score_bureau = spark.read.format("delta").load(PATH_SCORE_BUREAU)
print(f"✅ score_bureau_movel: {df_score_bureau.count():,} registros | {len(df_score_bureau.columns)} colunas")

# -----------------------------------------------------------------------------
# BOOKS DE FEATURES
# -----------------------------------------------------------------------------
try:
    df_book_recarga = spark.read.format("delta").load(PATH_BOOK_RECARGA_CMV)
    print(f"✅ book_recarga_cmv: {df_book_recarga.count():,} registros | {len(df_book_recarga.columns)} colunas")
    HAS_BOOK_RECARGA = True
except:
    print("⚠️ book_recarga_cmv: Não encontrado - será ignorado")
    HAS_BOOK_RECARGA = False

try:
    df_book_pagamento = spark.read.format("delta").load(PATH_BOOK_PAGAMENTO)
    print(f"✅ book_pagamento: {df_book_pagamento.count():,} registros | {len(df_book_pagamento.columns)} colunas")
    HAS_BOOK_PAGAMENTO = True
except:
    print("⚠️ book_pagamento: Não encontrado - será ignorado")
    HAS_BOOK_PAGAMENTO = False

try:
    df_book_faturamento = spark.read.format("delta").load(PATH_BOOK_FATURAMENTO)
    print(f"✅ book_faturamento: {df_book_faturamento.count():,} registros | {len(df_book_faturamento.columns)} colunas")
    HAS_BOOK_FATURAMENTO = True
except:
    print("⚠️ book_faturamento: Não encontrado - será ignorado")
    HAS_BOOK_FATURAMENTO = False

print("\n✅ Carregamento concluído")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 4, Finished, Available, Finished)

📊 Carregando tabelas...

✅ dados_cadastrais: 3,900,378 registros | 36 colunas
✅ telco: 1,367,104 registros | 77 colunas
✅ score_bureau_movel: 1,290,526 registros | 11 colunas
✅ book_recarga_cmv: 32,882,218 registros | 105 colunas
✅ book_pagamento: 12,633,614 registros | 157 colunas
✅ book_faturamento: 15,023,012 registros | 111 colunas

✅ Carregamento concluído


In [3]:
# =============================================================================
# REGISTRO DE VIEWS TEMPORÁRIAS
# =============================================================================

df_cadastro.createOrReplaceTempView("dados_cadastrais")
df_telco.createOrReplaceTempView("telco")
df_score_bureau.createOrReplaceTempView("score_bureau_movel")

if HAS_BOOK_RECARGA:
    df_book_recarga.createOrReplaceTempView("book_recarga_cmv")
    
if HAS_BOOK_PAGAMENTO:
    df_book_pagamento.createOrReplaceTempView("book_pagamento")
    
if HAS_BOOK_FATURAMENTO:
    df_book_faturamento.createOrReplaceTempView("book_faturamento")

print("✅ Views temporárias registradas")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 5, Finished, Available, Finished)

✅ Views temporárias registradas


In [4]:
# =============================================================================
# IDENTIFICAÇÃO DE COLUNAS A EXCLUIR
# =============================================================================

# Colunas de metadados que devem ser removidas de todas as tabelas
COLUNAS_METADADOS = [
    '_execution_id',
    '_data_inclusao',
    '_data_alteracao_silver',
    'DT_PROCESSAMENTO'  # Coluna de controle dos books
]

# Colunas de chave que só devem vir da tabela base (dados_cadastrais)
COLUNAS_CHAVE = ['NUM_CPF', 'SAFRA']

# Colunas duplicadas entre tabelas (manter apenas da base)
COLUNAS_DUPLICADAS_TELCO = ['FLAG_INSTALACAO', 'FPD', 'PROD', 'flag_mig2']
COLUNAS_DUPLICADAS_SCORE = ['FLAG_INSTALACAO', 'FPD', 'PROD', 'flag_mig2']

print("✅ Colunas de exclusão definidas")
print(f"   - Metadados: {COLUNAS_METADADOS}")
print(f"   - Chaves (só da base): {COLUNAS_CHAVE}")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 6, Finished, Available, Finished)

✅ Colunas de exclusão definidas
   - Metadados: ['_execution_id', '_data_inclusao', '_data_alteracao_silver', 'DT_PROCESSAMENTO']
   - Chaves (só da base): ['NUM_CPF', 'SAFRA']


In [5]:
# =============================================================================
# FUNÇÕES AUXILIARES
# =============================================================================

def get_columns_to_select(df, table_alias, exclude_cols, prefix=None):
    """
    Retorna lista de colunas formatadas para SELECT, excluindo as especificadas.
    
    Args:
        df: DataFrame Spark
        table_alias: Alias da tabela no SQL
        exclude_cols: Lista de colunas a excluir
        prefix: Prefixo opcional para renomear colunas
    
    Returns:
        String com colunas formatadas para SQL
    """
    cols = []
    for col in df.columns:
        if col.lower() not in [c.lower() for c in exclude_cols]:
            if prefix:
                cols.append(f"{table_alias}.{col} AS {prefix}_{col}")
            else:
                cols.append(f"{table_alias}.{col}")
    return cols

print("✅ Funções auxiliares carregadas")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 7, Finished, Available, Finished)

✅ Funções auxiliares carregadas


In [6]:
# =============================================================================
# CONSTRUÇÃO DINÂMICA DA QUERY
# =============================================================================

# -----------------------------------------------------------------------------
# 1. COLUNAS DA TABELA BASE (dados_cadastrais)
# -----------------------------------------------------------------------------
cols_cadastro = get_columns_to_select(
    df_cadastro, 
    'c', 
    COLUNAS_METADADOS
)
print(f"📊 dados_cadastrais: {len(cols_cadastro)} colunas selecionadas")

# -----------------------------------------------------------------------------
# 2. COLUNAS DA TABELA TELCO (excluir chaves e duplicadas)
# -----------------------------------------------------------------------------
exclude_telco = COLUNAS_METADADOS + COLUNAS_CHAVE + COLUNAS_DUPLICADAS_TELCO
cols_telco = get_columns_to_select(
    df_telco, 
    't', 
    exclude_telco
)
print(f"📊 telco: {len(cols_telco)} colunas selecionadas")

# -----------------------------------------------------------------------------
# 3. COLUNAS DO SCORE BUREAU (TARGET - variáveis resposta)
# -----------------------------------------------------------------------------
exclude_score = COLUNAS_METADADOS + COLUNAS_CHAVE + COLUNAS_DUPLICADAS_SCORE
cols_score = get_columns_to_select(
    df_score_bureau, 
    's', 
    exclude_score
)
# Renomear scores para identificar como TARGET
cols_score_renamed = []
for col in cols_score:
    if 'SCORE_' in col:
        cols_score_renamed.append(col.replace('s.SCORE_', 's.SCORE_') + ' AS TARGET_SCORE_' + col.split('SCORE_')[1])
    else:
        cols_score_renamed.append(col)
cols_score = cols_score_renamed
print(f"📊 score_bureau_movel (TARGET): {len(cols_score)} colunas selecionadas")

# -----------------------------------------------------------------------------
# 4. COLUNAS DO BOOK RECARGA CMV
# -----------------------------------------------------------------------------
if HAS_BOOK_RECARGA:
    exclude_recarga = COLUNAS_METADADOS + COLUNAS_CHAVE
    cols_recarga = get_columns_to_select(
        df_book_recarga, 
        'r', 
        exclude_recarga,
        prefix='REC'  # Prefixo para identificar origem
    )
    print(f"📊 book_recarga_cmv: {len(cols_recarga)} colunas selecionadas")
else:
    cols_recarga = []

# -----------------------------------------------------------------------------
# 5. COLUNAS DO BOOK PAGAMENTO
# -----------------------------------------------------------------------------
if HAS_BOOK_PAGAMENTO:
    exclude_pagamento = COLUNAS_METADADOS + COLUNAS_CHAVE
    cols_pagamento = get_columns_to_select(
        df_book_pagamento, 
        'p', 
        exclude_pagamento,
        prefix='PAG'  # Prefixo para identificar origem
    )
    print(f"📊 book_pagamento: {len(cols_pagamento)} colunas selecionadas")
else:
    cols_pagamento = []

# -----------------------------------------------------------------------------
# 6. COLUNAS DO BOOK FATURAMENTO
# -----------------------------------------------------------------------------
if HAS_BOOK_FATURAMENTO:
    exclude_faturamento = COLUNAS_METADADOS + COLUNAS_CHAVE
    cols_faturamento = get_columns_to_select(
        df_book_faturamento, 
        'f', 
        exclude_faturamento,
        prefix='FAT'  # Prefixo para identificar origem
    )
    print(f"📊 book_faturamento: {len(cols_faturamento)} colunas selecionadas")
else:
    cols_faturamento = []

# -----------------------------------------------------------------------------
# TOTAL DE COLUNAS
# -----------------------------------------------------------------------------
total_colunas = len(cols_cadastro) + len(cols_telco) + len(cols_score) + \
                len(cols_recarga) + len(cols_pagamento) + len(cols_faturamento) + 1  # +1 para DT_PROCESSAMENTO

print(f"\n📊 TOTAL DE COLUNAS NO SUPER BOOK: {total_colunas}")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 8, Finished, Available, Finished)

📊 dados_cadastrais: 33 colunas selecionadas
📊 telco: 68 colunas selecionadas
📊 score_bureau_movel (TARGET): 2 colunas selecionadas
📊 book_recarga_cmv: 102 colunas selecionadas
📊 book_pagamento: 154 colunas selecionadas
📊 book_faturamento: 108 colunas selecionadas

📊 TOTAL DE COLUNAS NO SUPER BOOK: 468


In [ ]:
# =============================================================================
# MONTAGEM DA QUERY SQL
# =============================================================================

# Concatenar todas as colunas
all_columns = cols_cadastro + cols_telco + cols_score + cols_recarga + cols_pagamento + cols_faturamento
all_columns.append("CURRENT_TIMESTAMP() AS DT_PROCESSAMENTO")

# Formatar colunas para SQL
columns_sql = ",\n        ".join(all_columns)

# Construir JOINs dinamicamente
joins_sql = """
    -- JOIN com Telco (variáveis de comportamento)
    LEFT JOIN telco t 
        ON c.NUM_CPF = t.NUM_CPF 
        AND c.SAFRA = t.SAFRA
    
    -- JOIN com Score Bureau (variáveis TARGET/resposta)
    LEFT JOIN score_bureau_movel s 
        ON c.NUM_CPF = s.NUM_CPF 
        AND c.SAFRA = s.SAFRA
"""

if HAS_BOOK_RECARGA:
    joins_sql += """
    -- JOIN com Book Recarga CMV (SAFRA agora é INT em ambos os lados)
    LEFT JOIN book_recarga_cmv r 
        ON c.NUM_CPF = r.NUM_CPF 
        AND c.SAFRA = r.SAFRA
"""

if HAS_BOOK_PAGAMENTO:
    joins_sql += """
    -- JOIN com Book Pagamento (SAFRA agora é INT em ambos os lados)
    LEFT JOIN book_pagamento p 
        ON c.NUM_CPF = p.NUM_CPF 
        AND c.SAFRA = p.SAFRA
"""

if HAS_BOOK_FATURAMENTO:
    joins_sql += """
    -- JOIN com Book Faturamento (SAFRA agora é INT em ambos os lados)
    LEFT JOIN book_faturamento f 
        ON c.NUM_CPF = f.NUM_CPF 
        AND c.SAFRA = f.SAFRA
"""

# Query final
query_super_book = f"""
-- =============================================================================
-- SUPER BOOK - FEATURE STORE CONSOLIDADA
-- =============================================================================
-- Granularidade: NUM_CPF + SAFRA
-- Destino: Gold.feature_store.clientes_consolidado
-- =============================================================================

SELECT 
        {columns_sql}

FROM dados_cadastrais c
{joins_sql}
"""

print("✅ Query SQL construída com sucesso")
print(f"\n📝 Preview da query (primeiros 2000 caracteres):")
print(query_super_book[:2000] + "...")

In [8]:
# =============================================================================
# EXECUÇÃO DA QUERY
# =============================================================================

print("⏳ Executando query do Super Book...")

df_super_book = spark.sql(query_super_book)

# Cache para otimização
df_super_book.cache()

# Contagens
total_registros = df_super_book.count()
total_colunas_final = len(df_super_book.columns)

print(f"\n✅ Super Book gerado com sucesso!")
print(f"📊 Total de registros: {total_registros:,}")
print(f"📊 Total de variáveis: {total_colunas_final}")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 10, Finished, Available, Finished)

⏳ Executando query do Super Book...

✅ Super Book gerado com sucesso!
📊 Total de registros: 3,900,378
📊 Total de variáveis: 468


In [9]:
# =============================================================================
# VALIDAÇÃO DO SCHEMA
# =============================================================================

print("📋 Schema do Super Book:")
print("=" * 70)

# Agrupar colunas por origem
colunas_por_origem = {
    'CADASTRO': [],
    'TELCO': [],
    'TARGET (SCORE_BUREAU)': [],
    'BOOK_RECARGA': [],
    'BOOK_PAGAMENTO': [],
    'BOOK_FATURAMENTO': [],
    'CONTROLE': []
}

for col in df_super_book.columns:
    if col.startswith('REC_'):
        colunas_por_origem['BOOK_RECARGA'].append(col)
    elif col.startswith('PAG_'):
        colunas_por_origem['BOOK_PAGAMENTO'].append(col)
    elif col.startswith('FAT_'):
        colunas_por_origem['BOOK_FATURAMENTO'].append(col)
    elif col.startswith('TARGET_') or col.startswith('SCORE_'):
        colunas_por_origem['TARGET (SCORE_BUREAU)'].append(col)
    elif col.startswith('var_'):
        colunas_por_origem['TELCO'].append(col)
    elif col == 'DT_PROCESSAMENTO':
        colunas_por_origem['CONTROLE'].append(col)
    else:
        colunas_por_origem['CADASTRO'].append(col)

for origem, colunas in colunas_por_origem.items():
    if colunas:
        print(f"\n📁 {origem} ({len(colunas)} colunas):")
        for col in colunas[:10]:  # Mostrar apenas 10 primeiras
            print(f"   - {col}")
        if len(colunas) > 10:
            print(f"   ... e mais {len(colunas) - 10} colunas")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 11, Finished, Available, Finished)

📋 Schema do Super Book:

📁 CADASTRO (9 colunas):
   - NUM_CPF
   - SAFRA
   - FLAG_INSTALACAO
   - FPD
   - PROD
   - flag_mig2
   - STATUSRF
   - DATADENASCIMENTO
   - CEP_3_digitos

📁 TELCO (92 colunas):
   - var_03
   - var_02
   - var_04
   - var_05
   - var_06
   - var_07
   - var_08
   - var_09
   - var_10
   - var_11
   ... e mais 82 colunas

📁 TARGET (SCORE_BUREAU) (2 colunas):
   - TARGET_SCORE_01
   - TARGET_SCORE_02

📁 BOOK_RECARGA (102 colunas):
   - REC_QTD_RECARGAS_TOTAL
   - REC_QTD_LINHAS_DISTINTAS
   - REC_QTD_DIAS_COM_RECARGA
   - REC_QTD_DIAS_ATIVOS
   - REC_QTD_RECARGAS_POR_DIA_ATIVO
   - REC_VLR_CREDITO_TOTAL
   - REC_VLR_CREDITO_MEDIO
   - REC_VLR_CREDITO_MAXIMO
   - REC_VLR_CREDITO_MINIMO
   - REC_VLR_CREDITO_DESVPAD
   ... e mais 92 colunas

📁 BOOK_PAGAMENTO (154 colunas):
   - PAG_QTD_PAGAMENTOS_TOTAL
   - PAG_QTD_CONTRATOS_DISTINTOS
   - PAG_QTD_FATURAS_DISTINTAS
   - PAG_QTD_DIAS_COM_PAGAMENTO
   - PAG_QTD_BANCOS_UTILIZADOS
   - PAG_QTD_FORMAS_PAGAMENTO_DISTI

In [10]:
# =============================================================================
# ESTATÍSTICAS DAS VARIÁVEIS TARGET
# =============================================================================

print("📊 Estatísticas das Variáveis TARGET (score_bureau_movel):")
print("=" * 70)

# Identificar colunas target
target_cols = [col for col in df_super_book.columns if 'TARGET_' in col or col in ['FPD', 'FLAG_INSTALACAO']]

if target_cols:
    df_super_book.select(target_cols).describe().show()
    
    # Distribuição do FPD (First Payment Default)
    print("\n📊 Distribuição do FPD (First Payment Default):")
    df_super_book.groupBy('FPD').count().orderBy('FPD').show()
else:
    print("⚠️ Nenhuma coluna TARGET identificada")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 12, Finished, Available, Finished)

📊 Estatísticas das Variáveis TARGET (score_bureau_movel):
+-------+------------------+-------------------+-----------------+-----------------+
|summary|   FLAG_INSTALACAO|                FPD|  TARGET_SCORE_01|  TARGET_SCORE_02|
+-------+------------------+-------------------+-----------------+-----------------+
|  count|           3900378|            2696621|          1281087|          1289950|
|   mean|0.6913742719295412|0.21272177291506666|586.9003986458375|627.5538416217682|
| stddev|0.4619263390952239|0.40923255289268035|57.48067205746377|96.07561713749614|
|    min|                 0|                  0|                0|                1|
|    max|                 1|                  1|              778|              917|
+-------+------------------+-------------------+-----------------+-----------------+


📊 Distribuição do FPD (First Payment Default):
+----+-------+
| FPD|  count|
+----+-------+
|NULL|1203757|
|   0|2122991|
|   1| 573630|
+----+-------+



In [11]:
# =============================================================================
# ANÁLISE DE COMPLETUDE (MISSING VALUES)
# =============================================================================

from pyspark.sql.functions import col, count, when, isnan, isnull

print("📊 Análise de Completude (% de valores não nulos):")
print("=" * 70)

# Calcular completude por grupo de colunas
total = df_super_book.count()

completude_por_origem = {}

for origem, colunas in colunas_por_origem.items():
    if colunas and origem != 'CONTROLE':
        # Calcular média de completude do grupo
        completude_media = 0
        for c in colunas:
            try:
                nao_nulos = df_super_book.filter(col(c).isNotNull()).count()
                completude_media += (nao_nulos / total) * 100
            except:
                pass
        completude_media = completude_media / len(colunas) if colunas else 0
        completude_por_origem[origem] = completude_media
        print(f"   {origem}: {completude_media:.1f}% completo")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 13, Finished, Available, Finished)

📊 Análise de Completude (% de valores não nulos):
   CADASTRO: 81.0% completo
   TELCO: 34.9% completo
   TARGET (SCORE_BUREAU): 33.0% completo
   BOOK_RECARGA: 71.1% completo
   BOOK_PAGAMENTO: 17.3% completo
   BOOK_FATURAMENTO: 21.7% completo


In [12]:
# =============================================================================
# AMOSTRA DOS DADOS
# =============================================================================

print("📋 Amostra dos dados (5 primeiros registros):")

# Selecionar algumas colunas representativas para visualização
colunas_amostra = ['NUM_CPF', 'SAFRA', 'PROD', 'FPD']

# Adicionar algumas colunas de cada origem
for origem, colunas in colunas_por_origem.items():
    if colunas and origem not in ['CADASTRO', 'CONTROLE']:
        colunas_amostra.extend(colunas[:2])  # 2 colunas de cada origem

df_super_book.select(colunas_amostra[:15]).show(5, truncate=False)

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 14, Finished, Available, Finished)

📋 Amostra dos dados (5 primeiros registros):
+-----------+------+----+----+------+------+---------------+---------------+----------------------+------------------------+------------------------+---------------------------+---------------------+----------------------+
|NUM_CPF    |SAFRA |PROD|FPD |var_03|var_02|TARGET_SCORE_01|TARGET_SCORE_02|REC_QTD_RECARGAS_TOTAL|REC_QTD_LINHAS_DISTINTAS|PAG_QTD_PAGAMENTOS_TOTAL|PAG_QTD_CONTRATOS_DISTINTOS|FAT_QTD_FATURAS_TOTAL|FAT_QTD_FATURAS_UNICAS|
+-----------+------+----+----+------+------+---------------+---------------+----------------------+------------------------+------------------------+---------------------------+---------------------+----------------------+
|777N8ZTXWZZ|202502|CMV |0   |37    |NULL  |NULL           |NULL           |3                     |2                       |NULL                    |NULL                       |NULL                 |NULL                  |
|777T8UZUYZZ|202412|CMV |0   |33    |NULL  |516            |549

In [ ]:
# =============================================================================
# SALVAMENTO NO LAKEHOUSE GOLD
# =============================================================================

print("⏳ Salvando Super Book no Lakehouse Gold...")

# dynamic partition overwrite e incompativel com overwriteSchema;
# como o consolidado faz overwrite total, usar static temporariamente
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "static")

df_super_book.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("SAFRA") \
    .option("overwriteSchema", "true") \
    .save(PATH_OUTPUT)

spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

print(f"\n✅ Super Book salvo com sucesso!")
print(f"📁 Destino: {PATH_OUTPUT}")

In [14]:
# =============================================================================
# VALIDAÇÃO DO SALVAMENTO
# =============================================================================

print("⏳ Validando salvamento...")

# Recarregar e validar
df_validacao = spark.read.format("delta").load(PATH_OUTPUT)

registros_salvos = df_validacao.count()
colunas_salvas = len(df_validacao.columns)

print(f"\n✅ Validação concluída:")
print(f"   - Registros salvos: {registros_salvos:,}")
print(f"   - Colunas salvas: {colunas_salvas}")
print(f"   - Match com geração: {'✅ OK' if registros_salvos == total_registros else '❌ DIVERGÊNCIA'}")

# Listar partições
print(f"\n📁 Partições (SAFRAs) disponíveis:")
df_validacao.select('SAFRA').distinct().orderBy('SAFRA').show(20)

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 16, Finished, Available, Finished)

⏳ Validando salvamento...

✅ Validação concluída:
   - Registros salvos: 3,900,378
   - Colunas salvas: 468
   - Match com geração: ✅ OK

📁 Partições (SAFRAs) disponíveis:
+------+
| SAFRA|
+------+
|202410|
|202411|
|202412|
|202501|
|202502|
|202503|
+------+



In [15]:
# =============================================================================
# REGISTRO NA FEATURE STORE (METADADOS)
# =============================================================================

from datetime import datetime

# Metadados do Super Book
metadata = {
    "nome": "super_book_clientes",
    "schema": "feature_store",
    "lakehouse": "Gold",
    "granularidade": "NUM_CPF + SAFRA",
    "total_registros": registros_salvos,
    "total_colunas": colunas_salvas,
    "particionamento": "SAFRA",
    "fontes": {
        "dados_cadastrais": len(cols_cadastro),
        "telco": len(cols_telco),
        "score_bureau_movel": len(cols_score),
        "book_recarga_cmv": len(cols_recarga) if HAS_BOOK_RECARGA else 0,
        "book_pagamento": len(cols_pagamento) if HAS_BOOK_PAGAMENTO else 0,
        "book_faturamento": len(cols_faturamento) if HAS_BOOK_FATURAMENTO else 0
    },
    "variaveis_target": target_cols,
    "dt_criacao": datetime.now().isoformat()
}

print("📋 Metadados do Super Book:")
print("=" * 70)
for key, value in metadata.items():
    print(f"   {key}: {value}")

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 17, Finished, Available, Finished)

📋 Metadados do Super Book:
   nome: super_book_clientes
   schema: feature_store
   lakehouse: Gold
   granularidade: NUM_CPF + SAFRA
   total_registros: 3900378
   total_colunas: 468
   particionamento: SAFRA
   fontes: {'dados_cadastrais': 33, 'telco': 68, 'score_bureau_movel': 2, 'book_recarga_cmv': 102, 'book_pagamento': 154, 'book_faturamento': 108}
   variaveis_target: ['FLAG_INSTALACAO', 'FPD', 'TARGET_SCORE_01', 'TARGET_SCORE_02']
   dt_criacao: 2026-01-15T00:23:16.042225


In [16]:
# =============================================================================
# LIMPEZA DE CACHE
# =============================================================================

df_super_book.unpersist()
print("✅ Cache liberado")
print("\n" + "=" * 70)
print("🏆 SUPER BOOK FINALIZADO COM SUCESSO!")
print("=" * 70)

StatementMeta(, 25913bcf-a55d-42cd-92a0-be21e43bd2d6, 18, Finished, Available, Finished)

✅ Cache liberado

🏆 SUPER BOOK FINALIZADO COM SUCESSO!


---

# 📖 Documentação do Book Cpnsolidado

## Resumo

| Métrica | Valor |
|---------|-------|
| **Tabela** | `feature_store.clientes_consolidado` |
| **Lakehouse** | Gold |
| **Granularidade** | NUM_CPF + SAFRA |
| **Particionamento** | SAFRA |

---

## Estrutura de Colunas por Origem

### 1. 📋 CADASTRO (dados_cadastrais)
Dados cadastrais e perfil básico do cliente.

| Coluna | Tipo | Descrição |
|--------|------|----------|
| NUM_CPF | string | CPF mascarado (chave) |
| SAFRA | int | Período YYYYMM (chave) |
| FLAG_INSTALACAO | int | Flag de instalação |
| FPD | int | First Payment Default |
| PROD | string | Produto (DTH, NET, CMV) |
| STATUSRF | string | Status Receita Federal |
| DATADENASCIMENTO | date | Data de nascimento |
| var_02 a var_25 | diversos | Variáveis de perfil |
| CEP_3_digitos | string | Região geográfica |

### 2. 📡 TELCO (telco)
Variáveis de comportamento de uso de telecomunicações.

| Coluna | Tipo | Descrição |
|--------|------|----------|
| var_26 a var_93 | decimal | Métricas de uso e perfil telco |

### 3. 🎯 TARGET (score_bureau_movel)
**Variáveis resposta para modelagem.**

| Coluna | Tipo | Descrição |
|--------|------|----------|
| TARGET_SCORE_01 | int | Score de bureau 1 |
| TARGET_SCORE_02 | int | Score de bureau 2 |

### 4. 💳 BOOK_RECARGA (ass_recarga_cmv_nova)
Variáveis derivadas de recargas (prefixo `REC_`).

### 5. 💰 BOOK_PAGAMENTO (book_recargas_pagamento)
Variáveis derivadas de pagamentos (prefixo `PAG_`).

### 6. 📄 BOOK_FATURAMENTO (book_faturamento)
Variáveis derivadas de faturamento (prefixo `FAT_`).

---

## Variáveis TARGET para Modelagem

O Super Book inclui as seguintes variáveis resposta do `score_bureau_movel`:

| Variável | Descrição | Uso |
|----------|-----------|-----|
| **FPD** | First Payment Default | Target binário para inadimplência |
| **TARGET_SCORE_01** | Score de bureau 1 | Target contínuo/ordinal |
| **TARGET_SCORE_02** | Score de bureau 2 | Target contínuo/ordinal |

---

## Exemplo de Uso

```python
# Carregar Super Book
df = spark.read.format("delta").load(
    "abfss://.../Tables/feature_store/clientes_consolidado"
)

# Filtrar por SAFRA
df_202401 = df.filter(df.SAFRA == 202401)

# Separar features e target
target_cols = ['FPD', 'TARGET_SCORE_01', 'TARGET_SCORE_02']
feature_cols = [c for c in df.columns if c not in target_cols + ['NUM_CPF', 'SAFRA']]

# Preparar para modelagem
X = df_202401.select(feature_cols)
y = df_202401.select('FPD')  # ou TARGET_SCORE_01/02
```

---

## Colunas Removidas

As seguintes colunas foram **intencionalmente excluídas**:

| Coluna | Motivo |
|--------|--------|
| `_execution_id` | Metadado de pipeline |
| `_data_inclusao` | Metadado de ingestão |
| `_data_alteracao_silver` | Metadado de ETL |
| `DT_PROCESSAMENTO` (dos books) | Evitar conflito |
| Chaves duplicadas | Mantidas apenas da tabela base |

---

## Notas Técnicas

1. **JOINs**: Todos LEFT JOIN para preservar base de dados_cadastrais
2. **Particionamento**: Por SAFRA para otimização de consultas
3. **Formato**: Delta Lake (ACID, time travel, schema evolution)
4. **Prefixos**: Colunas dos books têm prefixo para rastreabilidade
5. **Compatibilidade**: Spark SQL 3.x no Microsoft Fabric

---

**Fim da Documentação**
